In [1]:
import numpy as np
import pandas as pd


from sklearn.cross_validation import cross_val_score

import lightgbm as lgbm

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, HashingVectorizer
from sklearn import pipeline, preprocessing, feature_extraction
from sklearn.metrics import accuracy_score
from sklearn.linear_model import SGDClassifier
#import xgboost as xgb
import keras
from tqdm import tqdm
import os

/usr/local/lib/python3.5/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
MAX_SEQUENCE_LENGTH = 2000

MAX_NB_WORDS = 600000
EMBEDDING_DIM = 300

In [3]:
from gensim.models import KeyedVectors
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Flatten
from keras.layers.wrappers import TimeDistributed
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM
from keras.preprocessing.text import Tokenizer
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation
from keras.preprocessing.sequence import pad_sequences
from keras.layers.merge import concatenate
from keras.models import Model
from keras.layers import Embedding, Dense, Dropout, Reshape, Merge, BatchNormalization, TimeDistributed, Lambda
from keras.regularizers import l2
from keras.callbacks import Callback, ModelCheckpoint
from keras import backend as K

In [4]:
d = 'data/pan18-style-change-detection-training-dataset-2018-01-31/'
train_files = os.listdir(d)

truth_files = []
text_files = []
for i in train_files:
    if 'truth' in i:
        truth_files += [i]
    else:
        text_files += [i]
        
truth_files = np.sort(truth_files)
text_files = np.sort(text_files)


texts = []
truths = []
for i, j in zip(text_files, truth_files):
    file = open(d+i, 'r') 
    texts += [file.read()]
    
    file = open(d+j, 'r') 
    truths += [file.read()]
    
df = pd.DataFrame(texts)
df.columns = ['text']
df['coord'] = truths
df['coord'] = df['coord'].apply(lambda x: x.split('[')[1].split(']')[0])


d = 'data/pan18-style-change-detection-validation-dataset-2018-01-31/'
train_files = os.listdir(d)

truth_files = []
text_files = []
for i in train_files:
    if 'truth' in i:
        truth_files += [i]
    else:
        text_files += [i]
        
truth_files = np.sort(truth_files)
text_files = np.sort(text_files)


texts = []
truths = []
for i, j in zip(text_files, truth_files):
    file = open(d+i, 'r') 
    texts += [file.read()]
    
    file = open(d+j, 'r') 
    truths += [file.read()]
    
df_val = pd.DataFrame(texts)
df_val.columns = ['text']
df_val['coord'] = truths
df_val['coord'] = df_val['coord'].apply(lambda x: x.split('[')[1].split(']')[0])

In [5]:
df['text'] = df['text'].apply(lambda x: str(x))
df_val['text'] = df_val['text'].apply(lambda x: str(x))

In [7]:
y_train = df['coord'].apply(lambda x: len(x) > 0)
y_val = df_val['coord'].apply(lambda x: len(x) > 0)

In [8]:
train_texts1 = []
for i in df['text']:
    train_texts1.append(i)

    
test_texts1 = []
for i in df_val['text']:
    test_texts1.append(i)

In [9]:
tokenizer1 = Tokenizer(num_words=MAX_NB_WORDS)
tokenizer1.fit_on_texts(train_texts1 + test_texts1)

In [10]:
train_sequences_1 = tokenizer1.texts_to_sequences(train_texts1)
test_sequences_1 = tokenizer1.texts_to_sequences(test_texts1)

In [11]:
train_sequences_1 = pad_sequences(train_sequences_1, maxlen=MAX_SEQUENCE_LENGTH)
test_sequences_1 = pad_sequences(test_sequences_1, maxlen=MAX_SEQUENCE_LENGTH)

In [12]:
word2vec = KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)

In [13]:
word_index = tokenizer1.word_index
embedding_matrix = np.zeros((len(word_index) + 1, 300))
for word, i in word_index.items():
    if word in word2vec.vocab:
        embedding_matrix[i] = word2vec.word_vec(word)

In [14]:
embedding_layer = Embedding(embedding_matrix.shape[0],
                            embedding_matrix.shape[1],
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)

In [17]:
from keras.layers.merge import concatenate
from keras.layers.wrappers import Bidirectional 
def make_model():
    lstm_layer = Bidirectional(LSTM(300, dropout=0.2, recurrent_dropout=0.2, return_sequences=True))
    lstm_layer1 = Bidirectional(LSTM(300, dropout=0.2, recurrent_dropout=0.2, return_sequences=True))
    lstm_layer2 = Bidirectional(LSTM(300, dropout=0.2, recurrent_dropout=0.2, return_sequences=True))
    lstm_layer3 = Bidirectional(LSTM(300, dropout=0.2, recurrent_dropout=0.2, return_sequences=False))

    sequence_1_input = Input(shape=(2000,), dtype='int32')
    embedded_sequences_1 = embedding_layer(sequence_1_input)
    x1 = lstm_layer(embedded_sequences_1)
    x1 = lstm_layer1(x1)
    x1 = lstm_layer2(x1)
    x1 = lstm_layer3(x1)

    #merged = concatenate([x1, sequence_6_input, sequence_7_input])
#     merged = BatchNormalization()(x1)

    merged = Dense(512, activation='relu')(x1)
    merged = BatchNormalization()(merged)
    merged = Dropout(0.2)(merged)
    
    merged = Dense(256, activation='relu')(merged)
    merged = BatchNormalization()(merged)
    merged = Dropout(0.2)(merged)
    
    preds = Dense(1, activation='sigmoid')(merged)
    
    model = Model(inputs=[sequence_1_input], \
        outputs=preds)

    adam = keras.optimizers.Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)

    model.compile(loss='binary_crossentropy',
            optimizer=adam,
            metrics=['acc'])
    
    return model

In [18]:
model = make_model()
model_checkpoint = ModelCheckpoint('word_lstm_{}.h5'.format(0), save_best_only=True, save_weights_only=True)

model.fit([train_sequences_1],
      y_train, epochs=60,
      batch_size=10, shuffle=True,
      validation_data=([test_sequences_1], y_val), callbacks=[model_checkpoint])

Train on 2980 samples, validate on 1492 samples
Epoch 1/60
 860/2980 [=======>......................] - ETA: 54:49 - loss: 0.8995 - acc: 0.5081

KeyboardInterrupt: 